In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

/tmp/ipykernel_576108/2481774189.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
input_data_folder_base = Path("../data/preproc_data/")
output_data_folder_base = Path("../results/")
ardupilot_sub_folder = Path("./ardupilot")
measurement_sub_folder = Path("./measurements")

ardupilot_input_data_folder = Path.cwd() / input_data_folder_base / ardupilot_sub_folder 
measurement_input_data_folder = Path.cwd() / input_data_folder_base / measurement_sub_folder 
print(f"Reading raw pandas files from \n    {ardupilot_input_data_folder}")
print(f"Reading raw pandas files from \n    {measurement_input_data_folder}")

output_data_folder = Path.cwd() / output_data_folder_base  
print(f"Writing merged pandas file to \n    {output_data_folder}")

overall_collection_file_name = "overall_collection_2023"
key_parameter_file_name = "key_parameter_2023"

Reading raw pandas files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../data/preproc_data/ardupilot
Reading raw pandas files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../data/preproc_data/measurements
Writing merged pandas file to 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../results


In [3]:
columns = ['CO2(ppm)', 'airtemp in(degreeC)', 'humidity in(rH)', 'pressure in(mbar)',
           'airtemp out(degreeC)', 'humidity out(rH)', 'pressure out(mbar)', 
           'PAR(umol m-2s-1)', 'H2O temp(degreeC)']

df_summary = pd.DataFrame()

for file in list(ardupilot_input_data_folder.rglob("*.p")):
    df_ap=pd.read_pickle(file)
    print(f"{file.name} - duration {df_ap.index.min()} - {df_ap.index.max()}")

    # Add state information and file 
    # Merge data from measurement files
    df_ap['meas_running'] = False
    df_ap['corresponding_meas_file'] = ""
    df_ap[columns]=np.nan

    for file in list(measurement_input_data_folder.rglob("*ADC.p")):
        df_meas=pd.read_pickle(file)
        if (df_meas.index.max() < df_ap.index.max()) & (df_meas.index.min() > df_ap.index.min()):
            print(f"     {file.name} - duration {df_meas.index.min()} - {df_meas.index.max()}")
            df_ap.loc[(df_ap.index <= df_meas.index.max()) & (df_ap.index >= df_meas.index.min()), "meas_running"] = True
            df_ap.loc[(df_ap.index <= df_meas.index.max()) & (df_ap.index >= df_meas.index.min()), "corresponding_meas_file"] = file.name

            for index, row in df_meas.iterrows():
                id = df_ap.index.searchsorted(index)
                for entry in columns:
                    df_ap.iloc[id, df_ap.columns.get_loc(entry)] = row[entry]

    df_summary = pd.concat([df_summary, df_ap])

result_file = output_data_folder / Path(overall_collection_file_name+".p") 
print(f"... saving to {  Path(overall_collection_file_name+'.p')}")
df_summary.to_pickle(result_file)


040323-200000-AP.p - duration 2023-03-04 20:00:00 - 2023-03-11 20:17:04
     100323-121503-ADC.p - duration 2023-03-10 12:15:03 - 2023-03-10 12:20:04
     100323-161556-ADC.p - duration 2023-03-10 16:15:56 - 2023-03-10 16:20:57


     100323-150800-ADC.p - duration 2023-03-10 15:08:00 - 2023-03-10 15:38:40


     090323-154804-ADC.p - duration 2023-03-09 15:48:04 - 2023-03-09 15:53:05
     060323-122504-ADC.p - duration 2023-03-06 12:25:04 - 2023-03-06 12:30:00


     070323-125948-ADC.p - duration 2023-03-07 12:59:48 - 2023-03-07 13:04:48
     070323-143035-ADC.p - duration 2023-03-07 14:30:35 - 2023-03-07 15:01:06


     090323-111354-ADC.p - duration 2023-03-09 11:13:54 - 2023-03-09 11:18:55
     060323-134603-ADC.p - duration 2023-03-06 13:46:03 - 2023-03-06 13:51:04


...

     070323-140635-ADC.p - duration 2023-03-07 14:06:35 - 2023-03-07 14:11:36
     070323-131148-ADC.p - duration 2023-03-07 13:11:48 - 2023-03-07 13:42:19


     070323-151015-ADC.p - duration 2023-03-07 15:10:15 - 2023-03-07 15:44:27


     070323-124747-ADC.p - duration 2023-03-07 12:47:47 - 2023-03-07 12:52:48
     070323-115053-ADC.p - duration 2023-03-07 11:50:53 - 2023-03-07 12:21:24


     070323-112653-ADC.p - duration 2023-03-07 11:26:53 - 2023-03-07 11:31:50
     070323-111452-ADC.p - duration 2023-03-07 11:14:52 - 2023-03-07 11:19:53


... saving to overall_collection_2023.p
